Useful URLs

* Seasons page - http://www.espnfc.com/fifa-world-cup/4/scores?season=2014
* Individual match stats - http://www.espn.com/soccer/matchstats?gameId=383303
* line ups for the game - http://www.espn.com/soccer/lineups?gameId=383303

In [2]:
from selenium import webdriver

In [187]:
import datetime
import time

In [282]:
browser = webdriver.Firefox()
browser.get('http://www.espnfc.com/fifa-world-cup/4/scores?season=2014')

#### For a given year, iterate through the relevant dates

In [279]:
current_url = browser.current_url
print(current_url)

http://global.espn.com/football/scoreboard/_/league/fifa.world/date/20140612?src=com


In [156]:
def get_date(url):
    trim = url.split("?src=com")[0]
    date = trim.split("date/")[1]
    return date

In [186]:
def increment_date(date, delta):
    parsed_date = datetime.datetime.strptime(date,"%Y%m%d")
    new_date = parsed_date + datetime.timedelta(delta)
    new_date_string = str(new_date.year) + pad_date(new_date.month) + pad_date(new_date.day) 
    return new_date_string

In [174]:
def pad_date(day_or_month):
    if len(str(day_or_month)) == 1:
        return "0" + str(day_or_month)
    else:
        return str(day_or_month)

In [283]:
date_base_url = "http://global.espn.com/football/scoreboard/_/league/fifa.world/date/"
date = get_date(current_url)
for i in range(3):
    new_date = increment_date(date, i)
    print(new_date)
    browser.get(date_base_url + new_date)
    time.sleep(1)
    css_selector_match = 'article[class="scoreboard soccer fallback js-show"]'
    games = browser.find_elements_by_css_selector(css_selector_match)
    game_ids = get_game_ids(games)
    for game_id in game_ids:
        print(game_id)
        print(get_match_stats(game_id))

    

20140612
383303


NoSuchWindowException: Message: Unable to locate window


In [255]:
get_form_stats()

5
5


([{'datetime': '2014-06-06T19:00Z',
   'game_type': 'International Friendly',
   'score': '1-0',
   'team1': 'Brazil',
   'team2': 'Serbia'},
  {'datetime': '2014-06-03T19:00Z',
   'game_type': 'International Friendly',
   'score': '4-0',
   'team1': 'Brazil',
   'team2': 'Panama'},
  {'datetime': '2014-03-05T17:00Z',
   'game_type': 'International Friendly',
   'score': '0-5',
   'team1': 'South Africa',
   'team2': 'Brazil'},
  {'datetime': '2013-11-20T01:00Z',
   'game_type': 'International Friendly',
   'score': '2-1',
   'team1': 'Brazil',
   'team2': 'Chile'},
  {'datetime': '2013-11-17T00:30Z',
   'game_type': 'International Friendly',
   'score': '5-0',
   'team1': 'Brazil',
   'team2': 'Honduras'}],
 [{'datetime': '2014-06-06T23:00Z',
   'game_type': 'International Friendly',
   'score': '1-0',
   'team1': 'Croatia',
   'team2': 'Australia'},
  {'datetime': '2014-05-31T14:00Z',
   'game_type': 'International Friendly',
   'score': '2-1',
   'team1': 'Croatia',
   'team2': 'Mal

#### For a given date, get all games for the day

In [11]:
css_selector_match = 'article[class="scoreboard soccer fallback js-show"]'
games = browser.find_elements_by_css_selector(css_selector_match)

In [190]:
def get_game_ids(games):
    game_ids = []
    for game in games:
        game_ids.append(game.get_attribute("id"))
    return game_ids

In [13]:
game_ids

['383302', '383301', '383300']

#### For a given match, get relevant match statistics

In [108]:
match_stats_base_url = "http://www.espn.com/soccer/matchstats?gameId="
match_stats_lineups_base_url = "http://www.espn.com/soccer/lineups?gameId="

In [109]:
game_id = game_ids[0]

In [101]:
# score
def get_score():
    css_selector_scoreboard = 'div[class="competitors sm-score"]'
    scoreboard = browser.find_element_by_css_selector(css_selector_scoreboard)

    css_selector_home = 'div[class="team away "]' # mistake made by web developers
    home_scoreboard = scoreboard.find_element_by_css_selector(css_selector_home)
    home_team = home_scoreboard.find_element_by_css_selector('div[class="team-container"]').text
    home_score = int(home_scoreboard.find_element_by_css_selector('div[class="score-container"]').text)
    css_selector_away = 'div[class="team home "]'
    away_scoreboard = scoreboard.find_element_by_css_selector(css_selector_away)
    away_team = away_scoreboard.find_element_by_css_selector('div[class="team-container"]').text
    away_score = int(away_scoreboard.find_element_by_css_selector('div[class="score-container"]').text)

    return([home_team,home_score,away_team,away_score])

Mexico: 1, Cameroon: 0


In [102]:
# form
def get_form(home_team_last_five_games):
    form_stats = []
    for game in home_team_last_five_games:
        team1 = game.find_elements_by_class_name("long-name")[0].text
        team2 = game.find_elements_by_class_name("long-name")[1].text
        score = game.find_element_by_class_name("webview-internal").text
        datetime = game.find_element_by_css_selector('span[data-behavior="date_time"]').get_attribute("data-date")
        game_type = game.find_element_by_class_name("competition").text
        game_dict = {
            "team1": team1,
            "team2": team2,
            "score": score,
            "datetime": datetime,
            "game_type": game_type
        }
        form_stats.append(game_dict)
        
    return form_stats

In [254]:
def get_form_stats():
    id_form_home = "gamepackage-team-form-home"
    css_selector_form_game = 'tr'
    home_team_last_five_games = browser.find_element_by_id(id_form_home).find_element_by_css_selector('tbody').find_elements_by_css_selector(css_selector_form_game)
    print(len(home_team_last_five_games))
    home_form_stats = get_form(home_team_last_five_games)
    id_form_away = "gamepackage-team-form-away"
    away_team_last_five_games = browser.find_element_by_id(id_form_away).find_elements_by_css_selector(css_selector_form_game)
    print(len(away_team_last_five_games))
    away_form_stats = get_form(away_team_last_five_games)
    return home_form_stats, away_form_stats

In [105]:
# Match Stats
css_home_team_fouls = 'td[data-home-away="home"][data-stat="foulsCommitted"]'
css_away_team_fouls = 'td[data-home-away="away"][data-stat="foulsCommitted"]'
css_home_team_yellow_cards = 'td[data-home-away="home"][data-stat="yellowCards"]'
css_away_team_yellow_cards = 'td[data-home-away="away"][data-stat="yellowCards"]'
css_home_team_red_cards = 'td[data-home-away="home"][data-stat="redCards"]'
css_away_team_red_cards = 'td[data-home-away="away"][data-stat="redCards"]'
css_home_team_offsides = 'td[data-home-away="home"][data-stat="offsides"]'
css_away_team_offsides = 'td[data-home-away="away"][data-stat="offsides"]'
css_home_team_won_corners = 'td[data-home-away="home"][data-stat="wonCorners"]'
css_away_team_won_corners = 'td[data-home-away="away"][data-stat="wonCorners"]'
css_home_team_saves = 'td[data-home-away="home"][data-stat="saves"]'
css_away_team_saves = 'td[data-home-away="away"][data-stat="saves"]'


home_team_fouls = browser.find_element_by_css_selector(css_home_team_fouls).text
away_team_fouls = browser.find_element_by_css_selector(css_away_team_fouls).text
home_team_yellow_cards = browser.find_element_by_css_selector(css_home_team_yellow_cards).text
away_team_yellow_cards = browser.find_element_by_css_selector(css_away_team_yellow_cards).text
home_team_red_cards = browser.find_element_by_css_selector(css_home_team_red_cards).text
away_team_red_cards = browser.find_element_by_css_selector(css_away_team_red_cards).text
home_team_offsides = browser.find_element_by_css_selector(css_home_team_offsides).text
away_team_offsides = browser.find_element_by_css_selector(css_away_team_offsides).text
home_team_won_corners = browser.find_element_by_css_selector(css_home_team_won_corners).text
away_team_won_corners = browser.find_element_by_css_selector(css_away_team_won_corners).text
home_team_saves = browser.find_element_by_css_selector(css_home_team_saves).text
away_team_saves = browser.find_element_by_css_selector(css_away_team_saves).text

print(home_team, away_team)
print(home_team_fouls, away_team_fouls)
print(home_team_yellow_cards, away_team_yellow_cards)
print(home_team_red_cards, away_team_red_cards)
print(home_team_offsides, away_team_offsides)
print(home_team_won_corners, away_team_won_corners)
print(home_team_saves, away_team_saves)

Mexico Cameroon
11 12
1 1
0 0
5 3
2 5
3 1


In [106]:
# Possesions
css_home_team_possession = 'span[data-home-away="home"][data-stat="possessionPct"]'
css_away_team_possession = 'span[data-home-away="away"][data-stat="possessionPct"]'

home_team_possession = browser.find_element_by_css_selector(css_home_team_possession).text
away_team_possession = browser.find_element_by_css_selector(css_away_team_possession).text

print(home_team, away_team)
print(home_team_possession, away_team_possession)

Mexico Cameroon
58% 42%


In [107]:
# shots
css_home_team_shots = 'span[data-home-away="home"][data-stat="shotsSummary"]'
css_away_team_shots = 'span[data-home-away="away"][data-stat="shotsSummary"]'

home_team_shots = browser.find_element_by_css_selector(css_home_team_shots).text
away_team_shots = browser.find_element_by_css_selector(css_away_team_shots).text

print(home_team, away_team)
print(home_team_shots, away_team_shots)


Mexico Cameroon
9 (5) 10 (4)


In [276]:
def get_match_stats(game_id):
    # match stats
    browser.get(match_stats_base_url + game_id)
    browser.execute_script("window.stop();")
    print("page_loaded")
    home_form_stats, away_form_stats = get_form_stats()
    return([home_form_stats,away_form_stats])
    # lineup
#     browser.get(match_stats_lineups_base_url + game_id)
#     return match_stats

#### For a given match, get lineup

In [146]:
browser.get(match_stats_lineups_base_url + game_id)

In [147]:
team1 = browser.find_elements_by_class_name("table-accordion")[0]
team1_name = team1.find_element_by_css_selector('caption').text
print(team1_name)
team1_players = team1.find_element_by_css_selector('tbody').find_elements_by_css_selector("tr")
team1_player_names = list(map(lambda x: x.find_element_by_class_name("accordion-item").text, team1_players))
print(team1_player_names)

Mexico
['13 Guillermo Ochoa', '4 Rafael Márquez', '15 Héctor Moreno  ', '2 Maza', '7 Miguel Layún', '22 Paul Aguilar', '23 José Vázquez', '18 Andrés Guardado', '6 Héctor Herrera', '10 Giovani dos Santos', '19 Oribe Peralta  ']


In [148]:
team2 = browser.find_elements_by_class_name("table-accordion")[1]
team2_name = team2.find_element_by_css_selector('caption').text
print(team2_name)
team2_players = team2.find_element_by_css_selector('tbody').find_elements_by_css_selector("tr")
team2_player_names = list(map(lambda x: x.find_element_by_class_name("accordion-item").text, team2_players))
print(team2_player_names)

Cameroon
['16 Charles Itandje', '14 Aurélien Chedjou', "3 Nicolas N'Koulou", '6 Alex Song', '2 Benoît Assou-Ekotto', '4 Cedric Djeugoue', '18 Eyong Enoh', "17 Stéphane M'bia", '13 Maxim Choupo-Moting', '8 Benjamin Moukandjo', "9 Samuel Eto'o"]
